In [ ]:
import polars as pl
import pandas as pd
import scanpy as sc
from tqdm import tqdm


In [ ]:
meta = pd.read_csv('/workspaces/stimage/data/meta_all_gene.csv')
path = "/workspaces/stimage/data/raw"

In [10]:
data = meta.loc[(meta['species'] == 'human') & (meta['tissue'] == 'lung'), :]
data.index = range(len(data.index))

for i in range(len(data.index)):
    gene_exp_slide = pd.read_csv(
        f'{path}/{data.tech[i]}/gene_exp/{data.slide[i]}_count.csv',
        sep=',',
        nrows=1,
        index_col=0)
    if i == 0:
        gene_name_overlap = gene_exp_slide.columns
    else:
        gene_name_overlap = gene_name_overlap.intersection(
            gene_exp_slide.columns)
    print(i)


0
1
2
3
4


In [11]:
gene_exp_slide

,SAMD11,NOC2L,KLHL17,PLEKHN1,PERM1,HES4,ISG15,AGRN,RNF223,C1orf159,...,MT-ND2,MT-CO2,MT-ATP6,MT-CO3,MT-ND3,MT-ND4L,MT-ND4,MT-ND5,MT-ND6,MT-CYB
Human_Lung_10X_03292024_VisiumHD_s_008um_00301_00321-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,0.0,2.0


In [ ]:
data = data.iloc[:-1]
print(data)

In [6]:
# Function to read and process a single CSV file
def read_csv_file(file_path, image_id):
    df = pl.read_csv(file_path)
    df = df.with_columns(pl.lit(image_id).alias('image_id'))
    return df

In [7]:
# Read and concatenate all relevant CSV files
dfs = []
for index in tqdm(range(len(data)), desc="Reading slides"):
    slide = data['slide'].iloc[index]
    tech = data['tech'].iloc[index]

    # Construct the file path
    file_path = f'/workspaces/stimage/data/raw/{tech}/gene_exp/{slide}_count.csv'

    df = read_csv_file(file_path, slide)
    dfs.append(df)


Reading slides: 100%|██████████| 4/4 [00:13<00:00,  3.42s/it]


In [9]:
df

,SAMD11,NOC2L,KLHL17,PLEKHN1,PERM1,HES4,ISG15,AGRN,RNF223,C1orf159,TTLL10,TNFRSF18,TNFRSF4,SDF4,B3GALT6,C1QTNF12,UBE2J2,SCNN1D,ACAP3,PUSL1,INTS11,CPTP,TAS1R3,DVL1,MXRA8,AURKAIP1,CCNL2,ANKRD65,TMEM88B,VWA1,ATAD3C,ATAD3B,ATAD3A,TMEM240,SSU72,FNDC10,…,BRCC3,VBP1,RAB39B,CLIC2,TMLHE,SPRY3,VAMP7,IL9R,SRY,ZFY,TGIF2LY,PCDH11Y,AMELY,TBL1Y,PRKY,TSPY1,USP9Y,DDX3Y,UTY,TMSB4Y,NLGN4Y,AC007244.1,KDM5D,EIF1AY,DAZ2,MT-ND1,MT-ND2,MT-CO2,MT-ATP6,MT-CO3,MT-ND3,MT-ND4L,MT-ND4,MT-ND5,MT-ND6,MT-CYB,image_id
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
"""Human_Lung_10X_07132022_Visium…",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,"""Human_Lung_10X_07132022_Visium"""
"""Human_Lung_10X_07132022_Visium…",0.0,1.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0,2.0,0.0,5.0,0.0,0.0,2.0,0.0,4.0,1.0,5.0,1.0,0.0,2.0,0.0,5.0,14.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,2.0,0.0,…,2.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,23.0,24.0,18.0,37.0,8.0,31.0,36.0,9.0,26.0,30.0,"""Human_Lung_10X_07132022_Visium"""
"""Human_Lung_10X_07132022_Visium…",0.0,0.0,0.0,0.0,0.0,3.0,1.0,5.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,1.0,0.0,0.0,3.0,3.0,2.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,…,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0,16.0,10.0,11.0,19.0,14.0,6.0,12.0,13.0,9.0,11.0,8.0,"""Human_Lung_10X_07132022_Visium"""
"""Human_Lung_10X_07132022_Visium…",0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,2.0,0.0,4.0,0.0,0.0,0.0,0.0,3.0,0.0,3.0,1.0,0.0,0.0,0.0,2.0,8.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,3.0,0.0,…,0.0,0.0,0.0,0.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,28.0,44.0,63.0,53.0,81.0,13.0,70.0,91.0,18.0,24.0,26.0,"""Human_Lung_10X_07132022_Visium"""
"""Human_Lung_10X_07132022_Visium…",0.0,0.0,0.0,0.0,0.0,2.0,1.0,2.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,9.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,…,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,10.0,9.0,7.0,14.0,5.0,17.0,9.0,10.0,11.0,12.0,"""Human_Lung_10X_07132022_Visium"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Human_Lung_10X_07132022_Visium…",0.0,1.0,0.0,0.0,0.0,1.0,1.0,5.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,4.0,1.0,0.0,1.0,1.0,2.0,11.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,…,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,19.0,20.0,25.0,32.0,9.0,21.0,22.0,9.0,21.0,26.0,"""Human_Lung_10X_07132022_Visium"""
"""Human_Lung_10X_07132022_Visium…",0.0,4.0,0.0,2.0,0.0,1.0,3.0,12.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,1.0,1.0,7.0,1.0,9.0,1.0,0.0,5.0,0.0,5.0,40.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,…,1.0,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,47.0,54.0,152.0,77.0,170.0,20.0,167.0,160.0,32.0,57.0,63.0,"""Human_Lung_10X_07132022_Visium"""
"""Human_Lung_10X_07132022_Visium…",0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,…,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0,8.0,5.0,17.0,12.0,13.0,16.0,17.0,9.0,2.0,7.0,"""Human_Lung_10X_07132022_Visium"""


In [8]:
combined_df = pl.concat(dfs)

# Convert the combined Polars DataFrame to a pandas DataFrame
# (necessary for creating AnnData object)
pd_df = combined_df.to_pandas()

ShapeError: unable to append to a DataFrame of width 36603 with a DataFrame of width 18087

In [ ]:
index = df.columns[0]
data = df.drop(index).to_numpy()

In [ ]:
adata = sc.AnnData(X=data, obs=pd.DataFrame(index=df[index]), var=pd.DataFrame(index=df.columns[1:]))
print(adata)

In [ ]:
sc.pp.filter_cells(adata, min_genes=1)
sc.pp.filter_genes(adata, min_cells=1)

In [ ]:
sc.pp.normalize_total(adata)

In [ ]:
pd_df = df.to_pandas()
pd_df.to_hdf('gene_expression.h5', key='data', mode='w', complevel=5, complib='blosc')

In [ ]:
df.head()

In [ ]:
adata = sc.AnnData(pd_df)

In [ ]:
import h5py

with h5py.File('gene_expression.h5', 'r') as f:
    print(list(f['data']['axis0'].keys()))

In [ ]:
adata = sc.read_hdf('gene_expression.h5', key='data')